In [1]:
!rocm-smi



========================================= ROCm System Management Interface =========================================
=================================================== Concise Info ===================================================
Device  Node  IDs              Temp    Power  Partitions          SCLK    MCLK     Fan  Perf  PwrCap  VRAM%  GPU%  
              (DID,     GUID)  (Edge)  (Avg)  (Mem, Compute, ID)                                                   
0       4     0x740c,   57586  38.0°C  95.0W  N/A, N/A, 0         800Mhz  1600Mhz  0%   auto  560.0W  0%     0%    
1       5     0x740c,   45873  41.0°C  N/A    N/A, N/A, 0         800Mhz  1600Mhz  0%   auto  0.0W    0%     0%    
2       2     0x740c,   14571  41.0°C  94.0W  N/A, N/A, 0         800Mhz  1600Mhz  0%   auto  560.0W  0%     0%    
3       3     0x740c,   27432  36.0°C  N/A    N/A, N/A, 0         800Mhz  1600Mhz  0%   auto  0.0W    0%     0%    
4       8     0x740c,   30939  34.0°C  86.0W  N/A, N/A, 0         80

In [2]:
import torch

print(torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_properties(i))

True
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=63323962-3731-6162-6164-626232366438, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=61323937-6262-3735-3131-373961323732, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=36653564-3266-6336-3333-373563623430, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=36663136-3964-3862-6463-366333336561, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
tokenized_datasets_name = "dataset/tokenized"

In [5]:
from datasets import load_from_disk

tokenized_dataset = load_from_disk(tokenized_datasets_name)

tokenized_dataset

Loading dataset from disk:   0%|          | 0/713 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/80 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 98904411
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 10989380
    })
})

In [6]:
model_name = "wallacelw/ModBERTBr2"

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
from transformers import ModernBertForMaskedLM  

model = ModernBertForMaskedLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
)

model.to('cuda')

model

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(32768, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertUnpaddedRotaryEmbedding(dim=64, base=160000.0, scale_base=None)
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, 

In [9]:
from transformers import DataCollatorForLanguageModeling

# mask 30% of the tokens
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm = True,
    mlm_probability=0.3
) 

In [ ]:
from transformers import Trainer, TrainingArguments

evaluation_args = TrainingArguments(
    output_dir=f'evaluating/{model_name}',

    do_train=False,
    eval_strategy = "steps", 
    eval_steps=1_000,

    # eval_accumulation_steps=1,

    per_device_eval_batch_size=32,           # batch size for evaluation

    logging_strategy="steps",
    logging_steps=1_000,
    
    logging_dir=f'evaluation-logs/{model_name}',                # directory for storing logs
    report_to=["tensorboard"],

    save_strategy="steps",
    save_steps=1_000,                      # Save checkpoints every 100 steps
    save_total_limit=5,                  # Limit the total number of saved checkpoints

    fp16=True,    
)

evaluator = Trainer(
    model=model,   
    args=evaluation_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],         # Evaluation dataset
    data_collator=data_collator,
)

In [11]:
torch.cuda.empty_cache()

# Evaluate the model
eval_results = evaluator.evaluate()

/share/sw/ai/pytorch/2.5.1/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [12]:
# Print the results
print(eval_results)

{'eval_loss': 1.4970624446868896, 'eval_runtime': 27360.3303, 'eval_samples_per_second': 401.654, 'eval_steps_per_second': 1.569}
